In [1]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from scipy.stats import entropy
import numpy as np
import gc
import shapely
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

tqdm.pandas()

part = 7

bu = gpd.read_parquet(f'data/buurten_parts/buurten_c_{part}.parquet')

categorisation = pd.read_excel('data/categorisation.xlsx')
L0_cats = categorisation['L0 category'].unique()
L1_cats = categorisation['L1 category'].unique()

In [2]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

In [3]:
bu.head(3)

,buurtcode,buurtnaam,wijkcode,gemeentecode,gemeentenaam,IND_WBI,H2O,POSTCODE,DEK_PERC,OAD,...,total_amenities_1,L0_leibovici_1,L1_leibovici_1,L0_altieri_1,L1_altieri_1,total_amenities_2,L0_leibovici_2,L1_leibovici_2,L0_altieri_2,L1_altieri_2
9761,BU15980105,Handelskade en omgeving,WK159801,GM1598,Koggenland,1.0,NEE,1713,1.0,747.0,...,1.0,0.000,0.000,NaN,NaN,1.0,0.000,0.000,NaN,NaN
9762,BU15980106,Reigerlaan Obdam en omgeving,WK159801,GM1598,Koggenland,1.0,NEE,1713,1.0,747.0,...,4.0,2.375,2.375,2.751629,2.751629,4.0,2.375,2.375,2.751629,2.751629
9763,BU15980107,Dokter Lohmanstraat en omgeving,WK159801,GM1598,Koggenland,1.0,NEE,1713,1.0,784.0,...,0.0,0.000,0.000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.000000


In [4]:
for idx, row in tqdm(bu.iterrows(), total=bu.shape[0]):
    amenities = gpd.read_parquet(f"data/gm_amenities/amenities_{row['gemeentenaam']}.parquet")
    amenities = amenities[amenities.geometry.within(row.geometry)]
    for filter_i in [0, 1, 2]:
        
        # get and apply filter
        L0_filter, L1_filter = ec.getfilter(filter_i)
        
        amenities_f = amenities[~amenities.L0_category.isin(L0_filter)]
        if L1_filter:
            for key, value in L1_filter.items():
                amenities_f = amenities_f[
                    ~( (amenities_f.L0_category == key) & (amenities_f.L1_category.isin(value)) )
                ]
        
        bu.at[idx, f'L0_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L0_category.values)
        bu.at[idx, f'L1_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L1_category.values)
        
        for cat in L0_cats:
            bu.loc[idx, f'L1_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L0_category == cat])
        
        for cat in L1_cats:
            bu.loc[idx, f'L0_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L1_category == cat])

    del amenities
    del amenities_f
    gc.collect()

bu.to_parquet(f'data/buurten_parts/calculated/buurten_c_{part}.parquet') 

100%|██████████| 1394/1394 [17:01<00:00,  1.36it/s]
